In [1]:
import pandas as pd
import pyodbc 
import sqlalchemy
import time

driver = 'ODBC Driver 17 for SQL Server'
server = 'ig-seau-dev-reportingau-data.database.windows.net' 
database = 'AI' 
username = 'data_teamau' 
password = 'Tu5harTab!' 

server2 = '10.60.210.131' 
database2 = 'CCMData' 
username2 = 'sqladmin' 
password2 = 'Unl3a5h5ql' 



### Append to #8 Open Claim History.xlsx

In [2]:
# void
def insertIntoDb(dbName,dataList,truncate = False):
    try:
        conn = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'
                        ,autocommit=False)
        cursor = conn.cursor()
        if truncate:
            sql = f"Truncate table [{dbName}]"
            print("Truncation succeeded!!!!!\n")  
            
        sql = f'INSERT INTO [{dbName}] VALUES ({"?,"*(len(dataList[0]) - 1)}?)'
        print(sql)
        cursor.executemany(sql,dataList)
        cursor.commit()
        print("Insertion succeeded!!!!!\n")
    except Exception as e:
        print(e)


def deleteFromDb(dbName,month):
    try:
        conn = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'
                        ,autocommit=False)
        cursor = conn.cursor()
        sql = f"DELETE FROM {dbName} where [Month of Extract Date] = {month}"
        print(sql)
        cursor.execute(sql)
        cursor.commit()
        print("Deletion succeeded!!!!!\n")
    except Exception as e:
        print(e)

    
def overWriteExcelFile(inputPath,
                       outputPath,inputDataList,append = True):
    from pandas import ExcelWriter
    df1 = pd.read_excel(inputPath)
    outputColumns = df1.columns.values
    
    df2 = pd.DataFrame(inputDataList,
                 columns= outputColumns)
    if append:
        df3 = pd.concat([df1,df2],ignore_index = True)

    # Append to the current file.
        with ExcelWriter(outputPath,
                     engine="openpyxl",
                     mode='a') as writer:
            df3.to_excel(writer,sheet_name='Sheet1',index=False)
    else:
    # Overwrite the current file.
        with ExcelWriter(outputPath,
                     engine="openpyxl",
                     mode='w') as writer:
            df2.to_excel(writer,sheet_name='Sheet1',index=False)
        
        
def extractAggregatedDataFromSql(sqlquery,ser=server,db=database,user=username,pwd=password):
    l = []
    try:
        conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+ser
                      +';DATABASE='+db
                      +';UID='+user
                      +';PWD='+ pwd,
                        autocommit=False)
        cursor = conn.cursor()
        
        #print(sqlquery)
        cursor.execute(sqlquery)
        for row in cursor.fetchall():
            #print(tuple(row))
            l.append(tuple(row))
        print("Selection succeeded!!!!!\n")
        return l
    except Exception as e:
        print(e)
    
    
    
    

In [3]:
sqlQuery_7b = '''
Select [Client],[Consolidator],[IG Team],COUNT(Distinct [Claim Number]) as [Distinct count of Claim Number], 
DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0) [Month of Extract Date]  from
[dbo].[CDS.07a Fleet Monthly Performance] 
where [Status] = 'OPEN'
group by [Client],[Consolidator],[IG Team]

'''

sqlQuery_8b = '''
    Select a.[Business Area],a.[Division],a.[IG Group],a.[Insurer Name],count(a.[Claim No]) as [Count of Claim No],
    DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0)  [Month of Extract Date]  from
    (SELECT 
    CASE WHEN [Division] = 'SUA'
        OR [Insurer Name] = 'Allianz'
        OR [Insurer Name] = 'Inter Hannover'
        OR [Insurer Name] = 'Inter Hannover (Argis)'
        OR [Insurer Name] = 'RACQ - Famous'
        OR [Insurer Name] = 'Famous Motor Insurance' THEN 'NSW'

    WHEN [Division] = 'Property'
        OR [Division] = 'Builders Warranty'
        OR [Insurer Name] = 'Berkshire Hathaway Specialty Insurance' 
        OR [Insurer Name] = 'PD Insurance Agency Pty Ltd'
        OR [Insurer Name] = 'Progressive Direct Insurance'
        OR [Insurer Name] = 'IAL' THEN 'VIC'

    WHEN [Insurer Name] = 'CIL Insurance' 
            AND [Division] = 'Motor' THEN 'NSW'

    WHEN [Insurer Name] = 'Calliden Insurance Limited' 
            AND [Division] = 'Motor' THEN 'NSW'

    WHEN [Insurer Name] = 'Great Lakes Australia' 
            AND [Division] = 'Motor' THEN 'NSW'

    WHEN [Insurer Name] = 'Allied World' THEN 'NSW'

    WHEN [Insurer Name] = 'Insurance Box' THEN 'SPLIT - NSW LODGE'


    ELSE 'UNKNOWN'

    END [Business Area],

    [Division],
    CASE WHEN CHARINDEX('TP At Fault', [Loss Type]) > 0 OR 
                CHARINDEX('T/P At Fault', [Loss Type]) > 0OR
                CHARINDEX('T/P hit', [Loss Type]) > 0 OR
                CHARINDEX('TP hit', [Loss Type]) > 0 OR
                CHARINDEX('recovery', [Loss Type]) > 0OR
                CHARINDEX('Identified TP', [Loss Type]) > 0 OR
                CHARINDEX('BH - TP', [Loss Type]) > 0 THEN 'Recovery'

    WHEN  [Division] = 'Property' AND 
            [Recovery Estimate] > 0 THEN 'Recovery'

    ELSE    'Settlement' 

    END [IG Group],


    [Insurer Name], 
    [Claim No]

    from [CDS.08 Insurance Monthly Performance Summary]
    where [Settlement Date] IS NULL) a 
    group by  a.[Business Area],a.[Division],a.[IG Group],a.[Insurer Name]
'''

sqlQuery_7_Lodgements = '''
  Select FORMAT([Report Date],'yyyy-MM'),COUNT(DISTINCT [Claim Number]) 
  from [dbo].[CDS.07a Fleet Monthly Performance] 
  where 
  DATEDIFF(MONTH, [Report Date],DATEADD(MONTH, DATEDIFF(MONTH, -1, GETDATE())-1, -1)) >=0
  AND
  DATEDIFF(MONTH,[Report Date],DATEADD(MONTH, DATEDIFF(MONTH, -1, GETDATE())-1, -1)) < 12 
  group by FORMAT([Report Date],'yyyy-MM')
  order by FORMAT([Report Date],'yyyy-MM') ASC

'''

sqlQuery_7_Finalisations = '''
  Select FORMAT([Closed Date],'yyyy-MM'),COUNT(DISTINCT [Claim Number]) 
  from [dbo].[CDS.07a Fleet Monthly Performance] 
  where 
  DATEDIFF(MONTH, [Closed Date],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) >=0
  AND
  DATEDIFF(MONTH,[Closed Date],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) < 12 
  group by FORMAT([Closed Date],'yyyy-MM')
  order by FORMAT([Closed Date],'yyyy-MM') ASC
'''


sqlQuery_7_Opens = '''
  Select 
  FORMAT([Month of Extract Date],'yyyy-MM'),
  SUM([Distinct count of Claim Number])
  from [dbo].[CDS.07b] 
  where 
  DATEDIFF(MONTH, [Month of Extract Date],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) >=0
  AND
  DATEDIFF(MONTH,[Month of Extract Date],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) < 12 
  group by FORMAT([Month of Extract Date],'yyyy-MM')
  order by FORMAT([Month of Extract Date],'yyyy-MM') ASC
'''


sqlQuery_8_Lodgements = '''
  Select FORMAT([Registered Date],'yyyy-MM'),COUNT(DISTINCT [Claim No]) 
  from [dbo].[CDS.08 Insurance Monthly Performance Summary] 
  where 
  DATEDIFF(MONTH, [Registered Date],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) >=0
  AND
  DATEDIFF(MONTH,[Registered Date],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) < 12 
  AND [Division] in ('Motor','Property','SUA')
  group by FORMAT([Registered Date],'yyyy-MM')
  order by FORMAT([Registered Date],'yyyy-MM') ASC
'''

sqlQuery_8_Finalisations = '''
  Select FORMAT([Settlement Date],'yyyy-MM'),COUNT(DISTINCT [Claim No]) 
  from [dbo].[CDS.08 Insurance Monthly Performance Summary] 
  where 
  DATEDIFF(MONTH, [Settlement Date],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) >=0
  AND
  DATEDIFF(MONTH,[Settlement Date],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) < 12 
  AND [Division] in ('Motor','Property','SUA')
  group by FORMAT([Settlement Date],'yyyy-MM')
  order by FORMAT([Settlement Date],'yyyy-MM') ASC
'''


sqlQuery_8_Opens = '''
  Select FORMAT([Month of Extract Date],'yyyy-MM'),SUM( [Count of Claim No]) 
  from [dbo].[CDS.08b] 
  where 
  DATEDIFF(MONTH, [Month of Extract Date],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) >=0
  AND
  DATEDIFF(MONTH,[Month of Extract Date],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) < 12 
  AND [Division] in ('Motor','Property','SUA')
  group by FORMAT([Month of Extract Date],'yyyy-MM')
  order by FORMAT([Month of Extract Date],'yyyy-MM') ASC

  
'''


sqlQuery_27_Lodgements = '''
  Select FORMAT([Input Date],'yyyy-MM'),COUNT(DISTINCT [Afs Ref]) 
  from [dbo].[CDS.27 Figtree Lodgements] 
  where 
  DATEDIFF(MONTH, [Input Date],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) >=0
  AND
  DATEDIFF(MONTH,[Input Date],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) < 12 
  AND [Extract date] = (Select MAX([Extract date]) from [dbo].[CDS.27 Figtree Lodgements])
  group by FORMAT([Input Date],'yyyy-MM')
  order by FORMAT([Input Date],'yyyy-MM') ASC
'''

sqlQuery_27_Finalisations = '''
  Select FORMAT([Finalised],'yyyy-MM'),COUNT(DISTINCT [Afs Ref]) 
  from [dbo].[CDS.27 Figtree Lodgements] 
  where 
  DATEDIFF(MONTH, [Finalised],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) >=0
  AND
  DATEDIFF(MONTH,[Finalised],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) < 12 
  AND [Re-open Status] = 'F'
  AND [Extract date] = (Select MAX([Extract date]) from [dbo].[CDS.27 Figtree Lodgements])
  AND [TP Recov Status] in
  ('*** - Invalid Code ***',
  '*** - Invalid Code ***',
  'AFS Paid to AFS Trust Account',
  'CAT Closed Unable to Locate TP',
  'CAT REC Closed Unable to Locate TP',
  'DFR Not Economical to Proceed',
  'DFR REC Not Economical to Proceed',
  'FRA Funds Recovered - Agency',
  'FRI Funds Recovered - Insurer',
  'LD Liability disputed',
  'LD REC Liability disputed',
  'NA Not Applicable',
  'RC Funds Recovered',
  'RC Funds Recovered - Direct',
  'SC SET Settlement Complete',
  'SC Settlement Complete')
  group by FORMAT([Finalised],'yyyy-MM')
  order by FORMAT([Finalised],'yyyy-MM') ASC

'''


sqlQuery_27_Opens = '''

  Select FORMAT([Extract Date],'yyyy-MM'),COUNT( [Afs Ref]) 
  from [dbo].[CDS.27 Figtree Lodgements] 
  where 
  DATEDIFF(MONTH, [Extract Date],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) >=0
  AND
  DATEDIFF(MONTH,[Extract Date],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) < 12 

  AND ([Re-open Status] ='C'
  OR  [TP Recov Status] IN ('AIA REC Further details',
    'CA-SP REC POL sent to TPD, follow up req',
    'CA-SP REC POL sent to TPD, follow up required',
    'CA-SP Settlement Pending',
    'CAU REC Collection agent: awaiting update',
    'CDS Recovery Pending',
    'FL Proof of Loss Issued',
    'FL REC POL sent to TPI, follow up requir',
    'FL REC POL sent to TPI, follow up required',
    'FNL First Notification of Loss',
    'FNL REC First LOD sent',
    'IPA Installment Plan Accepted',
    'IPA Instalment Plan Accepted',
    'IPA REC Follow up weekly instalments',
    'LAD REC Claim referred - Collection Agen',
    'LAD REC Claim referred - Collection Agent',
    'LAI Referred to RCL',
    'LGL REC Claim referred - Legal',
    'LU Liability Undecided',
    'REG REC Awaiting rego/licence search',
    'SKIP REC Awaiting skip trace',
    'SL Notice of Intention to Sue Issued',
    'SL REC NOITS sent to TPD follow up requi',
    'SL REC NOITS sent to TPD follow up required',
    'SLI REC NOITS sent to TPI follow up requ',
    'SLI REC NOITS sent to TPI follow up required',
    'SR SET Settlement Required',
    'SR Settlement Required')
  OR [Repair Status] IN ('000 Unspecified',
      'ACA Awaiting Client Authority',
      'ACI All Claim Information not Received',
      'AR At Repairer',
      'ARA Awaiting Repair Authority',
      'ASCF Awaiting Signed Claim Form',
      'CC Client Complaint',
      'DF Daily follow up',
      'RCAU Repairs Completed, Awaiting Updated',
      'RCAU Repairs Completed, Awaiting Updated Inv',
      'RD Repairs Delayed',
      'RPI Repairs Completed, Awaiting Invoice',
      'RS Referral Sent, Awaiting Estimate',
      'TL Total Loss Awaiting Financial Info.'))


  group by FORMAT([Extract Date],'yyyy-MM') 
  order by FORMAT([Extract Date],'yyyy-MM') ASC
  
'''


sqlQuery_13_Files = '''
  Select FORMAT([INVOICE DATE],'yyyy-MM') ,COUNT([Claim Number]) 
  from [dbo].[CDS.13b] 
  where 
  DATEDIFF(MONTH, [INVOICE DATE],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) >=0
  AND
  DATEDIFF(MONTH,[INVOICE DATE],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) < 12 
  AND (Left([Invoice Code],1) <> 'E')
  group by FORMAT([INVOICE DATE],'yyyy-MM') 
  order by FORMAT([INVOICE DATE],'yyyy-MM') ASC 
  
'''

sqlQuery_13_Commissions = '''
  Select FORMAT([INVOICE DATE],'yyyy-MM') ,ROUND(SUM(
    CASE WHEN [Calc_RecoveryCommissonPaid]='TRUE' THEN -[Invoice Amount]*[Calc_Commission rates] ELSE 0 END),0) [Commision]
  from 
  (Select *,CASE WHEN  [Fasttrack Consolidator] = 'AB Group' 
    OR [Fasttrack Consolidator] = 'Australia Post' AND [Invoice Date]>= '2018-07-09'
    OR [Fasttrack Consolidator] = 'Cleanaway Waste Management Limited' 
    OR [Fasttrack Consolidator] = 'Europcar' 
    OR [Fasttrack Consolidator] = 'Holden Ltd' AND CHARINDEX('Holden',[Fasttrack Client]) > 0 AND [Invoice Date]>= '2019-01-01'
    OR [Fasttrack Consolidator] = 'Lease Plan Total Cover' AND datediff(day, [Letter Of Demand], [Invoice Date]) <=91 
    OR [Fasttrack Consolidator] = 'Star Track Express Pty Ltd' 
    OR [Fasttrack Consolidator] = 'Tasty Trucks Pty Ltd' 
    OR [Fasttrack Consolidator] = 'Tourism Holdings Limited' AND [Invoice Date]>= '2018-06-12' 
    OR [Fasttrack Consolidator] = 'Zurich' 
    OR CHARINDEX('Holden Leasing',[Fasttrack Consolidator]) > 0
    OR [Fasttrack Consolidator] = 'Interleasing' 
    OR [Fasttrack Consolidator] = 'Wesfarmers Limited' 
    AND ([Fasttrack Client] = 'Kmart Tyre & Auto' OR [Fasttrack Client] = 'Kmart' OR [Fasttrack Client] = 'Target') 
    OR ([Fasttrack Consolidator] = 'Coles Group' OR [Fasttrack Consolidator] = 'Coles Group Ltd' 
    OR [Fasttrack Consolidator] = 'Wesfarmers Limited') 
        AND ([Fasttrack Client] = 'Coles/Bilo' 
        OR [Fasttrack Client] = 'Coles Online' 
        OR [Fasttrack Client] = 'Services Coles' 
        OR [Fasttrack Client] = 'Coles Trailers' 
        OR [Fasttrack Client] = 'Liquor Group' 
        OR [Fasttrack Client] = 'Coles Express' 
        OR [Fasttrack Client] = 'Coles Supermarkets' 
        OR [Fasttrack Client] = 'Coles Online - NSW' 
        OR [Fasttrack Client] = 'Coles Online - QLD' 
        OR [Fasttrack Client] = 'Coles Online - SA' 
        OR [Fasttrack Client] = 'Coles Online - VIC' 
        OR [Fasttrack Client] = 'Coles Online - WA' 
        OR [Fasttrack Client] = 'CGL Liquor Group' 
        OR [Fasttrack Client] = 'CGL Retail Support') AND [Invoice Date]>= '2017-01-01' 
    OR [Fasttrack Consolidator] = 'SurePlan Australia' AND [Fasttrack Client] = 'Tasty Trucks Pty Ltd' 
    OR [Fasttrack Consolidator] = 'SurePlan Australia' AND [Fasttrack Client] = 'Landmark' AND [Invoice Date]>= '2019-01-01' 
    OR [Fasttrack Consolidator] = 'SurePlan Australia' AND [Fasttrack Client] = 'Kmart Tyre & Auto Service' AND [Invoice Date]>= '2018-12-01' 
    OR [Fasttrack Consolidator] = 'SmartFleet' AND [Invoice Date]>= '2019-03-01'
    OR [Fasttrack Consolidator] = 'Summit Fleet Leasing and Management' AND [Invoice Date]>= '2019-03-01'
    THEN 'TRUE'
    ELSE 'FALSE'
    END 
    [Calc_RecoveryCommissonPaid],

  CASE WHEN  ([Fasttrack Consolidator] = 'AB Group' OR  
  [Fasttrack Consolidator] = 'Cleanaway Waste Management Limited' OR  
  [Fasttrack Consolidator] = 'Coles Group Ltd' OR  
  [Fasttrack Consolidator] = 'Holden Leasing'  OR
  [Fasttrack Consolidator] = 'Interleasing' OR 
  [Fasttrack Consolidator] = 'Star Track Express Pty Ltd') OR
  ([Fasttrack Consolidator] = 'Wesfarmers Limited' AND  ([Fasttrack Client] = 'Kmart' OR
  [Fasttrack Client] = 'Kmart Tyre & Auto' OR 
  [Fasttrack Client] = 'Target')) OR

   
  ([Fasttrack Consolidator] = 'Australia Post' AND [Invoice Date] >= '2018-07-09') OR 
  ([Fasttrack Consolidator] = 'Holden Ltd' AND [Invoice Date] >= '2019-01-01') OR 
  (([Fasttrack Consolidator] = 'SmartFleet' AND [Invoice Date] >= '2019-05-01') OR 
  ([Fasttrack Consolidator] = 'Summit Fleet Leasing and Management') AND [Invoice Date] >= '2019-03-01') OR 
  ([Fasttrack Consolidator] = 'SurePlan Australia' AND 
      ([Fasttrack Client] = 'Kmart Tyre & Auto Service' AND [Invoice Date] >= '2018-12-01') OR
      ([Fasttrack Client] = 'Landmark'  AND [Invoice Date] >= '2019-01-01')) OR

  ([Fasttrack Consolidator] = 'Wesfarmers Limited' OR 
      [Fasttrack Consolidator] = 'Coles Group Ltd' OR [Fasttrack Consolidator] = 'Coles Group') AND 
      ([Fasttrack Client] = 'Coles Online - NSW' OR [Fasttrack Client] = 'Coles Online - QLD' OR 
      [Fasttrack Client] = 'Coles Online - SA' OR 
      [Fasttrack Client] = 'Coles Online - VIC' OR [Fasttrack Client] = 'Coles Online - QLD' OR 
      [Fasttrack Client] = 'Coles Online - WA' OR [Fasttrack Client] = 'Coles Express' OR 
      [Fasttrack Client] = 'Coles Supermarkets' OR [Fasttrack Client] =  'Coles Trailers' OR 
      [Fasttrack Client] = 'Coles/Bilo' OR  [Fasttrack Client] = 'Liquor Group' OR 
      [Fasttrack Client] = 'Services Coles' OR [Fasttrack Client] = 'Coles Online' OR
      [Fasttrack Client] = 'Coles Express' OR [Fasttrack Client] = 'Coles Supermarkets' OR  
      [Fasttrack Client] =  'Coles Trailers' OR [Fasttrack Client] = 'Coles/Bilo' OR
      [Fasttrack Client] = 'Liquor Group' OR [Fasttrack Client] = 'Services Coles' OR [Fasttrack Client] = 'Coles Online' OR
       [Fasttrack Client] = 'Coles Online - QLD' OR [Fasttrack Client] = 'CGL Liquor Group' OR [Fasttrack Client] = 'CGL Retail Support')

  THEN 0.075

  WHEN [Fasttrack Consolidator] = 'Zurich' 
  THEN 0.05

  WHEN [Fasttrack Consolidator] = 'SurePlan Australia' AND  [Fasttrack Client] = 'Tasty Trucks Pty Ltd' 
  THEN 0.06

  WHEN [Fasttrack Consolidator] = 'Tourism Holdings Limited' AND [Invoice Date] >= '2018-06-12'
  THEN 0.055

  --- New Europcar Commission of 12.5% to start for all Lodgements after 1st August 2019 as per email from James White on 23rd July 2019
  WHEN [Fasttrack Consolidator] = 'Europcar' AND [Report Date] < '2019-08-01'
  THEN 0.08

  WHEN [Fasttrack Consolidator] = 'Europcar' AND [Report Date] >= '2019-08-01'
  THEN 0.125


  --- New LPTC Commission of 7.5% to start from Receipt date 1st July 2019 as per email from Neville Read 27th June 2019
  WHEN [Fasttrack Consolidator] = 'Lease Plan Total Cover' AND [Invoice Date]>=[Letter Of Demand] AND [Invoice Date]<=dateadd(DAY,90,[Letter Of Demand])
  AND [Invoice Date]<='2019-06-30'
  THEN 0.05

  WHEN [Fasttrack Consolidator] = 'Lease Plan Total Cover' AND [Invoice Date]>=[Letter Of Demand] AND [Invoice Date]<=dateadd(DAY,90,[Letter Of Demand])
  THEN 0.075

  --- Elders - New SurePlan Client added with 10% Commission
  WHEN [Fasttrack Client] = 'Elders' AND [Fasttrack Consolidator] = 'SurePlan Australia'
  THEN 0.1

  ELSE 0 END [Calc_Commission rates]

  
  
  from [dbo].[CDS.13b] 
  where 
  DATEDIFF(MONTH, [INVOICE DATE],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) >=0
  AND
  DATEDIFF(MONTH,[INVOICE DATE],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) < 12 
  AND (Left([Invoice Code],1) <> 'E')
  AND NOT ([Invoice Code] = 'C4' OR [Invoice Code] = 'CA' ))
  a 

  group by FORMAT([INVOICE DATE],'yyyy-MM') 
  order by FORMAT([INVOICE DATE],'yyyy-MM') ASC

'''


sqlQuery_15_Files = '''
  Select FORMAT([Recovery Date],'yyyy-MM'),COUNT([Claim Number])
  from [dbo].[CDS.15]
  where 
  DATEDIFF(MONTH, [Recovery Date],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) >=0
  AND
  DATEDIFF(MONTH,[Recovery Date],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) < 12 
  AND [Claims Consultant IG Employee] = 'Yes'
  group by FORMAT([Recovery Date],'yyyy-MM')
  order by FORMAT([Recovery Date],'yyyy-MM') ASC
  
'''

sqlQuery_15_Commissions = '''
  Select FORMAT([Recovery Date],'yyyy-MM') ,ROUND(SUM(CASE WHEN 
    (
      (
      ([Lawyer Name] IS NULL AND [Recovery Company] IS NULL)
    OR [Lawyer Assigned Date] >= [Recovery Date]
    OR [Recovery Assigned Date] >= [Recovery Date]
    )
    AND [Recovery Code] = 'Proof of Loss' 
    AND [Claims Consultant IG Employee] = 'Yes' 
    AND ([Insurer] = 'Allianz' 
      OR [Insurer] = 'RACQ - Famous'
      OR [Insurer] = 'Inter Hannover' 
      OR [Insurer] = 'Inter Hannover (Argis)' 
      OR [Insurer]= 'HDI Global Specialty (One Underwriting)' 
      OR [Insurer] = 'Allied World')
  )

  OR 
    (
      (
      ([Lawyer Name] IS NULL AND [Recovery Company] IS NULL)
    OR [Lawyer Assigned Date] >= [Recovery Date]
    OR [Recovery Assigned Date] >= [Recovery Date]
    )

   AND [Recovery Code] = 'Proof of Loss'
   AND [Claims Consultant IG Employee] = 'Yes' 
   AND [Insurer] = 'Great Lakes Australia'
   AND (LEFT([Claim Number],4) = 'GLAR' 
        OR LEFT([Claim Number],4) = 'GLFB')
   )

    THEN 0.07 * [Recovery Received] ELSE 0 END),0)
  from [dbo].[CDS.15]
  where 
  DATEDIFF(MONTH, [Recovery Date],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) >=0
  AND
  DATEDIFF(MONTH,[Recovery Date],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) < 12 
  AND [Recovery Code] = 'Proof of Loss'
  AND [Claims Consultant IG Employee] = 'Yes'
  group by FORMAT([Recovery Date],'yyyy-MM') 
  order by FORMAT([Recovery Date],'yyyy-MM')  ASC
'''


sqlQuery_17_Files = '''
 Select FORMAT([Trans'n Dt],'yyyy-MM') ,COUNT([Afs Ref])

  from [dbo].[CDS.17]
  where 
  DATEDIFF(MONTH, [Trans'n Dt],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) >=0
  AND
  DATEDIFF(MONTH,[Trans'n Dt],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) < 12 

  group by FORMAT([Trans'n Dt],'yyyy-MM') 
  order by FORMAT([Trans'n Dt],'yyyy-MM')  ASC
  
'''

sqlQuery_17_Commissions = '''
  Select FORMAT([Trans'n Dt],'yyyy-MM'),
  ROUND(SUM(CASE WHEN [Client] ='FA' AND 0.15*Abs([Amount]) <= 250 THEN 250
    WHEN [Client] ='FA' AND 0.15*Abs([Amount]) > 250 THEN 0.15*Abs([Amount]) 
    WHEN 0.15*Abs([Amount]) >= 150  THEN 0.15*Abs([Amount]) ELSE 150 END),0)
  from [dbo].[CDS.17]
  where 
  DATEDIFF(MONTH, [Trans'n Dt],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) >=0
  AND
  DATEDIFF(MONTH,[Trans'n Dt],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) < 12 

  group by FORMAT([Trans'n Dt],'yyyy-MM') 
  order by FORMAT([Trans'n Dt],'yyyy-MM')  ASC

'''

sqlQuery_21_Fleet = '''
SELECT  [MidnightStartDate],
sub.a as [Calls],
case when sub.a = 0 THEN 0 ELSE 1.0*sub.b/sub.a  END [% SLA]

 FROM 
 (Select FORMAT([MidnightStartDate],'yyyy-MM')  as [MidnightStartDate] ,sum([QueueOffered]) a ,sum([QueueServiceCount]) b 
 FROM [CCMData].[dbo].[QueuePerformanceByPeriodStats]

 where 
   DATEDIFF(MONTH, [MidnightStartDate],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) >=0
  AND
  DATEDIFF(MONTH,[MidnightStartDate],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) < 12
  AND
  [QueueName] in 
('AI-NLC',
'AustExistClaim',
'AustNewClaim',
'Clms Svcs Rntl Clms',
'Gen Fleet Catrastrophe',
'Gen Fleet Catrastrophe_resil',
'Gnrc Flet Lgmt',
'Gnrc Flet Othr',
'Hire Car',
'Interleasing-Exst',
'MelbW-Exist',
'MelbW-Rep',
'Rcvr Flet',
'Rental',
'Sureplan3rd',
'SurePlan-Exist',
'SurePlan-Gen',
'SurePlan-New',
'SurePlan-Rep',
'Tech Asmt',
'Tech Gnrl',
'Tech Insr',
'Warriewood Catastrophe',
'Warriewood Catastrophe_resil',
'WW Exst',
'WW New',
'WW Rental',
'WW Settlement',
'WW Settlement Res',
'zzCleanExistClaimzz',
'zzCleanGeneralzz',
'zzCleanNewClaimzz',
'zzCleanRepairerAssesszz',
'zzCleanThirdPartyzz',
'zzColes-Onlinezz',
'zzColes-Trailerszz',
'zzDept-Dfnc-Newzz',
'zzDept-Dfnc-Othrzz',
'zzFleetPart-3rdzz',
'zzFleetPart-Existzz',
'zzFleetPart-Genzz',
'zzFleetPart-Newzz',
'zzFleetPart-Repzz',
'zzFleetPlus-3rdzz',
'zzFleetPlus-Existzz',
'zzFleetPlus-Genzz',
'zzFleetPlus-Newzz',
'zzFleetPlus-Repzz',
'zzHold-Lease-Existzz',
'zzHold-Lease-Newzz',
'zzHolden-Excesszz',
'zzHolden-Existzz',
'zzHolden-Newzz',
'zzInterleasing-Exstzz',
'zzInterleasing-Newzz',
'zzLandmarkzz',
'zzLeasePlanzz',
'zzMelbW-3rdzz',
'zzMelbW-Existzz',
'zzMelbW-Genzz',
'zzMelbW-Newzz',
'zzMelbW-Repzz',
'zzMurrGoulb-Existzz',
'zzMurrGoulb-Newzz',
'zzReceptionzz',
'zzShell-Existzz',
'zzShell-Newzz',
'zzStewartzz',
'zzSummit-Existzz',
'zzSummit-Newzz',
'zzToyotazz',
'zzWeb-Smart-3rdzz',
'zzWeb-Smart-Existzz',
'zzWeb-Smart-Genzz',
'zzWeb-Smart-Repzz',
'zzWeb-Smart-Newzz')

  GROUP BY FORMAT([MidnightStartDate],'yyyy-MM') 
  
  ) sub

 order by [MidnightStartDate] ASC


'''

sqlQuery_21_Insurance = '''

SELECT  [MidnightStartDate],
sub.a as [Calls],
case when sub.a = 0 THEN 0 ELSE 1.0*sub.b/sub.a  END [% SLA]

 FROM 
 (Select FORMAT([MidnightStartDate],'yyyy-MM')  as [MidnightStartDate] ,sum([QueueOffered]) a ,sum([QueueServiceCount]) b 
 FROM [CCMData].[dbo].[QueuePerformanceByPeriodStats]

 where 
   DATEDIFF(MONTH, [MidnightStartDate],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) >=0
  AND
  DATEDIFF(MONTH,[MidnightStartDate],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) < 12
  AND
  [QueueName] in 
('Allied World Exist',
'Allied World New',
'Argis-Exst',
'Argis-New',
'Armada Cat',
'Armada Catastrophe_Resil',
'Armada Exst',
'Armada New',
'Aust-Rlnc-Exst',
'Aust-Rlnc-New',
'Calliden-Claims',
'Chubb Exst',
'Chubb New',
'Dawes UW Catastrophe',
'Dawes UW Catastrophe_resil',
'Daws Undw Exst',
'Daws Undw New',
'Famous Catastrophe',
'Famous Catastrophe_resil',
'Famous Exist',
'Famous New',
'Gen Insurance Cat',
'Gen Insurance Cat_resil',
'Gnrc Insr Exst',
'Gnrc Insr New',
'Gnrc Insr Othr',
'Hlrd Assg',
'Hlrd TP Exst',
'Hlrd TP New',
'Ibox Cat_Resil',
'Ibox Catastrophe',
'iBox Exst',
'iBox New',
'IF Exist',
'IF New',
'Mansions Exst',
'Mansions New',
'MtrClAss',
'One Underwriting Exist',
'One Underwriting New',
'Poncho Cnts Exst',
'Poncho Cnts New',
'Poncho Mtr Exst',
'Poncho Mtr New',
'Porsche Cat',
'Porsche Cat_resil',
'Porsche Exst',
'Porsche New',
'Prog Drct Exst',
'Prog Drct New',
'Progressive-Exist',
'Progressive-New',
'RACWA-Exist',
'RACWA-New',
'Rcvr Insr',
'Rep-Property',
'Repr Mgmt',
'Repr Mgmt Hlrd',
'SUA Exst',
'SUA New',
'Syd-MtrClaims',
'Zurich',
'Zurich-Booking',
'zzArgis',
'zzIAL Mtr Exstzz',
'zzIAL Mtr Newzz',
'zzIAL Prop Exstzz',
'zzIAL Prop Newzz')

  GROUP BY FORMAT([MidnightStartDate],'yyyy-MM') 
  
  ) sub

 order by [MidnightStartDate] ASC


'''

sqlQuery_21_TP = '''

SELECT  [MidnightStartDate],
sub.a as [Calls],
case when sub.a = 0 THEN 0 ELSE 1.0*sub.b/sub.a  END [% SLA]

 FROM 
 (Select FORMAT([MidnightStartDate],'yyyy-MM') as [MidnightStartDate] ,sum([QueueOffered]) a ,sum([QueueServiceCount]) b 
 FROM [CCMData].[dbo].[QueuePerformanceByPeriodStats]

 where 
   DATEDIFF(MONTH, [MidnightStartDate],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) >=0
  AND
  DATEDIFF(MONTH,[MidnightStartDate],DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0 )) < 12
  AND
  [QueueName] in 
('Suncorp Drct Bkng',
'Suncorp Drct Hire',
'Suncorp Exst',
'Suncorp New',
'Suncorp Othr')
  GROUP BY FORMAT([MidnightStartDate],'yyyy-MM') 
  
  ) sub

 order by [MidnightStartDate] ASC


'''

## Upload new [CDS.13b] and [CDS.15] (Cuz it changes daily)

In [6]:
new_13b_file = pd.read_excel('G:/Reporting & Insights/Tableau Server Source Files/#13b Fast Track Invoice Recoveries.xlsx')
new_15_file = pd.read_excel('G:/Reporting & Insights/Tableau Server Source Files/#15 Insurance Client Recoveries CFT.xlsx')
engine = sqlalchemy.create_engine(f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver}')

# replace will automatically truncate the table 
new_13b_file.to_sql('CDS.13b',engine,if_exists='replace')
new_15_file.to_sql('CDS.15',engine,if_exists='replace')

# If the [CDS.12] is empty, we need to upload using the historical file (up to last month)

#new_12_file = pd.read_excel('G:/Reporting & Insights/Tableau Server Source Files/#12 Regional Pack data.xlsx',sheet_name='Final Raw Data')
#new_12_file.to_sql('CDS.12',engine,if_exists='replace',index=False)

In [37]:
# Insurance
lodgements_8 = extractAggregatedDataFromSql(sqlquery=sqlQuery_8_Lodgements)
finalisations_8 = extractAggregatedDataFromSql(sqlquery=sqlQuery_8_Finalisations)
opens_8 = extractAggregatedDataFromSql(sqlquery=sqlQuery_8_Opens)
callsAndSLA_21_Insurance = extractAggregatedDataFromSql(sqlquery=sqlQuery_21_Insurance,ser=server2,db=database2,user=username2,pwd=password2)
files_15 = extractAggregatedDataFromSql(sqlquery=sqlQuery_15_Files)
commisions_15 = extractAggregatedDataFromSql(sqlquery=sqlQuery_15_Commissions)

#Fleet
lodgements_7 = extractAggregatedDataFromSql(sqlquery=sqlQuery_7_Lodgements)
finalisations_7 = extractAggregatedDataFromSql(sqlquery=sqlQuery_7_Finalisations)
opens_7 = extractAggregatedDataFromSql(sqlquery=sqlQuery_7_Opens)
callsAndSLA_21_Fleet = extractAggregatedDataFromSql(sqlquery=sqlQuery_21_Fleet,ser=server2,db=database2,user=username2,pwd=password2)
files_13 = extractAggregatedDataFromSql(sqlquery=sqlQuery_13_Files)
commisions_13 = extractAggregatedDataFromSql(sqlquery=sqlQuery_13_Commissions)

#FigTree
lodgements_27 = extractAggregatedDataFromSql(sqlquery=sqlQuery_27_Lodgements)
finalisations_27 = extractAggregatedDataFromSql(sqlquery=sqlQuery_27_Finalisations)
opens_27 = extractAggregatedDataFromSql(sqlquery=sqlQuery_27_Opens)
callsAndSLA_21_TP = extractAggregatedDataFromSql(sqlquery=sqlQuery_21_TP,ser=server2,db=database2,user=username2,pwd=password2)
files_17 = extractAggregatedDataFromSql(sqlquery=sqlQuery_17_Files)
commisions_17 = extractAggregatedDataFromSql(sqlquery=sqlQuery_17_Commissions)



Selection succeeded!!!!!

Selection succeeded!!!!!

Selection succeeded!!!!!

Selection succeeded!!!!!

Selection succeeded!!!!!

Selection succeeded!!!!!

Selection succeeded!!!!!

Selection succeeded!!!!!

Selection succeeded!!!!!

Selection succeeded!!!!!

Selection succeeded!!!!!

Selection succeeded!!!!!

Selection succeeded!!!!!

Selection succeeded!!!!!

Selection succeeded!!!!!

Selection succeeded!!!!!

Selection succeeded!!!!!

Selection succeeded!!!!!



In [38]:
for a,b,c in zip(callsAndSLA_21_Insurance,callsAndSLA_21_Fleet,callsAndSLA_21_TP):
    print (a,b,c)

('2019-03', 3826, Decimal('0.814427600627')) ('2019-03', 10432, Decimal('0.639570552147')) ('2019-03', 4767, Decimal('0.466540801342'))
('2019-04', 3285, Decimal('0.860882800608')) ('2019-04', 9327, Decimal('0.658625495872')) ('2019-04', 4131, Decimal('0.413459210844'))
('2019-05', 3612, Decimal('0.862403100775')) ('2019-05', 10715, Decimal('0.722071861875')) ('2019-05', 5066, Decimal('0.300434267666'))
('2019-06', 2759, Decimal('0.843421529539')) ('2019-06', 8499, Decimal('0.715613601600')) ('2019-06', 4004, Decimal('0.385114885114'))
('2019-07', 3104, Decimal('0.870811855670')) ('2019-07', 9185, Decimal('0.823843222645')) ('2019-07', 4702, Decimal('0.438962143768'))
('2019-08', 2608, Decimal('0.866947852760')) ('2019-08', 8260, Decimal('0.847094430992')) ('2019-08', 4569, Decimal('0.669949660757'))
('2019-09', 2415, Decimal('0.798757763975')) ('2019-09', 8202, Decimal('0.811753230919')) ('2019-09', 4312, Decimal('0.775510204081'))
('2019-10', 2417, Decimal('0.836988001654')) ('2019-1

In [39]:
fleet_lodgements_Total = [(x[0],x[1] + y[1]) for x,y in zip(lodgements_7,lodgements_27)]
fleet_finalisations_Total = [(x[0],x[1] + y[1]) for x,y in zip(finalisations_7,finalisations_27)]
fleet_opens_Total = [(x[0],x[1] + y[1]) for x,y in zip(opens_7,opens_27)]
fleet_files_Total = [(x[0],x[1] + y[1]) for x,y in zip(files_13,files_17)]
fleet_commision_Total = [(x[0],x[1] + y[1]) for x,y in zip(commisions_13,commisions_17)]

In [40]:
fleet_lodgements_Total

[('2019-03', 2870),
 ('2019-04', 2493),
 ('2019-05', 2736),
 ('2019-06', 2458),
 ('2019-07', 2711),
 ('2019-08', 2481),
 ('2019-09', 2402),
 ('2019-10', 2658),
 ('2019-11', 2755),
 ('2019-12', 2325),
 ('2020-01', 3225),
 ('2020-02', 2601)]

In [41]:
fleet_finalisations_Total

[('2019-03', 2592),
 ('2019-04', 2506),
 ('2019-05', 3348),
 ('2019-06', 3202),
 ('2019-07', 3116),
 ('2019-08', 2920),
 ('2019-09', 2313),
 ('2019-10', 3264),
 ('2019-11', 2724),
 ('2019-12', 2275),
 ('2020-01', 2489),
 ('2020-02', 2758)]

In [42]:
fleet_opens_Total

[('2019-03', 11145),
 ('2019-04', 11240),
 ('2019-05', 10758),
 ('2019-06', 10018),
 ('2019-07', 9599),
 ('2019-08', 9229),
 ('2019-09', 9324),
 ('2019-10', 8753),
 ('2019-11', 8849),
 ('2019-12', 8888),
 ('2020-01', 9628),
 ('2020-02', 9591)]

In [43]:
fleet_files_Total

[('2019-03', 1038),
 ('2019-04', 1043),
 ('2019-05', 981),
 ('2019-06', 899),
 ('2019-07', 1097),
 ('2019-08', 865),
 ('2019-09', 1123),
 ('2019-10', 879),
 ('2019-11', 885),
 ('2019-12', 757),
 ('2020-01', 1175),
 ('2020-02', 839)]

In [44]:
fleet_commision_Total

[('2019-03', 71068.0),
 ('2019-04', 49712.0),
 ('2019-05', 56102.0),
 ('2019-06', 52682.0),
 ('2019-07', 73719.0),
 ('2019-08', 85318.0),
 ('2019-09', 77809.0),
 ('2019-10', 60335.0),
 ('2019-11', 67976.0),
 ('2019-12', 64421.0),
 ('2020-01', 58052.0),
 ('2020-02', 39953.0)]

In [45]:
fleet = [(x[0],'Fleet',x[1],y[1],z[1],d[1],d[2],a[1],b[1]) for x,y,z,d,a,b in zip(fleet_lodgements_Total,
                                                      fleet_finalisations_Total,
                                                      fleet_opens_Total,
                                                      callsAndSLA_21_Fleet,          
                                                      fleet_files_Total,
                                                      fleet_commision_Total)]

insurance =[(x[0],'Insurance',x[1],y[1],z[1],d[1],d[2],a[1],b[1]) for x,y,z,d,a,b in zip(lodgements_8,
                                                                             finalisations_8,
                                                                             opens_8,
                                                                             callsAndSLA_21_Insurance,
                                                                             files_15,
                                                                             commisions_15)]

TpPro = [(d[0],'TP Pro',0,0,0,d[1],d[2],0,0) for d in callsAndSLA_21_TP]


In [46]:
fleetDF = pd.DataFrame(fleet,columns=['Month','Portfolio','Lodgements','Finalisations','Open Claims',	'Calls','SLA','Files Recovered','Recovery Commission' ])
insuranceDF = pd.DataFrame(insurance,columns=['Month','Portfolio','Lodgements','Finalisations','Open Claims',	'Calls','SLA','Files Recovered','Recovery Commission' ])
TpProDF = pd.DataFrame(TpPro,columns=['Month','Portfolio','Lodgements','Finalisations','Open Claims',	'Calls','SLA','Files Recovered','Recovery Commission' ])


In [47]:
from datetime import datetime
regional = pd.concat([fleetDF,insuranceDF,TpProDF],axis = 0,ignore_index=True)
regional['Month'] = regional['Month'].apply(lambda x:datetime.strptime(x, '%Y-%m'))
regional.sort_values('Month',inplace=True)

# Extract last month's data and insert into [CDS.12]
import datetime
today = datetime.date.today()
first = today.replace(day=1)
lastMonth = first - datetime.timedelta(days=1)
toSubtract = int(lastMonth.strftime('%d')) -1
firstOfLastMonth = (lastMonth - datetime.timedelta(days = toSubtract))

regional_last_month = regional.loc[regional['Month'] == firstOfLastMonth,]

regional_last_month

In [48]:
# Generate new records then insert into [CDS.08b]. Override the local excel at the end
#new_open_claim_7 = extractAggregatedDataFromSql(sqlQuery_7b)
#new_open_claim_8 = extractAggregatedDataFromSql(sqlQuery_8b)

In [49]:
# Insert the new data to [CDS.08b],[CDS.07b],[CDS.12]
#insertIntoDb("CDS.07b",new_open_claim_7)
#insertIntoDb("CDS.08b",new_open_claim_8)
insertIntoDb("CDS.12",regional.values.tolist(),truncate= True)

Truncation succeeded!!!!!

INSERT INTO [CDS.12] VALUES (?,?,?,?,?,?,?,?,?)
Insertion succeeded!!!!!



### If you run multiple times (duplicated), just delete with


Delete  from [CDS.12] where [Month] = DATEADD(MONTH, DATEDIFF(MONTH, 0, GETDATE())-1, 0) 

OR

Delete  from [CDS.12] where [Month] = 'yyyy-mm-01'

### in SSMS. 

Thus, the [CDS.12] will just keep appending but never truncate. Because the previous code always output the rolling 12 months.

overWriteExcelFile('G:/Reporting & Insights/Tableau Server Source Files/#7b Fast Track Open Claims History.xlsx',
                   'G:/Reporting & Insights/Tableau Server Source Files/#7b Fast Track Open Claims History New.xlsx',
                  new_open_claim_7)

overWriteExcelFile('G:/Reporting & Insights/Tableau Server Source Files/#8b CFT Open Claims History.xlsx',
                   'G:/Reporting & Insights/Tableau Server Source Files/#8b CFT Open Claims History New.xlsx',
                  new_open_claim_8)


In [50]:

# Output the rolling 12 months in new Excel file. 
# (Because we use [CDS.12] for tableau. Need to overwrite the data in the old #12 excel file to keep it updated for archieve purpose.)
overWriteExcelFile('G:/Reporting & Insights/Tableau Server Source Files/#12 Regional Pack data.xlsx',
                   'G:/Reporting & Insights/Tableau Server Source Files/#12 Regional Pack data New.xlsx',
                  regional,append= False)
